In [1]:
# Good practice to have the libraries set here. 

import findspark
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import DataFrameWriterV2
#Lets check SPARK, where exactly it is installed.
findspark.init()
findspark.find()

'C:\\Softwares\\Spark_installtion\\spark-3.4.1-bin-hadoop3'

In [2]:
#lets initiate spark session.  I am using Spark 3.4.1 so  there is no need to start the spark context. 
#It is available in the system itself. 

spark = (
            SparkSession
                .builder
                .appName("DBConnection" ) # any name for yourself. 
                .master("local[*]") # Why this? well, we are running in local mode. 
                .config("spark.jars","./JARS/sqljdbc_12.4/enu/jars/spark-mssql-connector_2.12-1.2.0.jar") # this is required to read the data from SQL SERVER
                .getOrCreate() 
        )

spark # lets just check if it is set. 

In [ ]:
###This is required when I have to stop the spark and restart the sessions. #####

#sc = spark.sparkContext
#sc.stop()
#spark.stop()
#print("server stopped")

In [3]:
server_name = "jdbc:sqlserver://localhost:1433"
database_name = "SALES50M"
url = server_name + ";" + "databaseName=" + database_name + ";encrypt=true;trustServerCertificate=true;"
table_name = "[SALES50M].[dbo].[SALES]"
username = "Root"
password = "Root@123" # Please specify password here

In [4]:
#added the JAR file in the system ( wherever spark is installed) or add it in the dependecy in the DatabRicks library

Sales_DF = spark.read \
        .format("com.microsoft.sqlserver.jdbc.spark") \
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password)\
        .load()


In [16]:
Sales_DF.head()

Row(ID=44896043, ORDERID=8977488, ORDERDETAILID=44896048, DATE_=datetime.datetime(2023, 3, 30, 0, 0), USERID=64416, USERNAME_='cey_dikcinar@fakehotmail.com', NAMESURNAME='Ceylan Cansu DİKÇINAR', STATUS_=1, ITEMID=10790, ITEMCODE='4704', ITEMNAME='SERVE CREATIVE VERSATIL KALEM 0.7', AMOUNT=4.0, UNITPRICE=7.2, PRICE=7.5, TOTALPRICE=30.0, CATEGORY1='EV', CATEGORY2='KITAP-DERGI-KIRTASIYE', CATEGORY3='KIRTASIYE', CATEGORY4='KIRTASIYE GERECLERI', BRAND='KIRTASIYELER', USERGENDER='K', USERBIRTHDATE=datetime.date(1953, 11, 28), REGION='Marmara', CITY='İstanbul', TOWN='ÇATALCA', DISTRICT='ÖRCÜNLÜ KÖYÜ', ADDRESSTEXT='ÖRCÜNLÜ KÖYÜ MAH. ALPTEKİN SOKAK  34540  ÇATALCA/İSTANBUL', ADDRESSID=None)

In [40]:
Sales_DF.columns

['ID',
 'ORDERID',
 'ORDERDETAILID',
 'DATE_',
 'USERID',
 'USERNAME_',
 'NAMESURNAME',
 'STATUS_',
 'ITEMID',
 'ITEMCODE',
 'ITEMNAME',
 'AMOUNT',
 'UNITPRICE',
 'PRICE',
 'TOTALPRICE',
 'CATEGORY1',
 'CATEGORY2',
 'CATEGORY3',
 'CATEGORY4',
 'BRAND',
 'USERGENDER',
 'USERBIRTHDATE',
 'REGION',
 'CITY',
 'TOWN',
 'DISTRICT',
 'ADDRESSTEXT',
 'ADDRESSID']

In [25]:
PRODUCT_DF = Sales_DF.select(
 'ITEMID',
 'ITEMCODE',
 'ITEMNAME',
 'UNITPRICE',
 'CATEGORY1',
 'CATEGORY2',
 'CATEGORY3',
 'CATEGORY4',
 'BRAND').distinct()

In [26]:
 PRODUCT_DF.count()

27000

In [32]:
#sometimes you will see error if you use the complete stream. So just JDBC works in the higher versions of spark. 
PRODUCT_DF.write \
    .format("jdbc") \ 
    .mode("overwrite") \
    .option("url", url) \
    .option("dbtable", "ProductMaster_STG") \
    .option("user", username) \
    .option("password", password) \
    .save()

In [43]:
#lets break the table into smaller tables to have the smaller set. lets have the users here . 
User_DF = Sales_DF.select(
 'USERID',
 'USERNAME_',
 'NAMESURNAME',
 'USERGENDER',
 'USERBIRTHDATE').distinct()

User_DF.count()

100217

In [46]:
#sometimes you will see error if you use the complete stream. So just JDBC works in the higher versions of spark. 
User_DF.write \
    .format("jdbc") \
    .mode("overwrite") \
    .option("url", url) \
    .option("dbtable", "UserMaster_STG") \
    .option("user", username) \
    .option("password", password) \
    .save()

In [49]:
Address_DF = Sales_DF.select(
'ADDRESSID',
 'REGION',
 'CITY',
 'TOWN',
 'DISTRICT',
 'ADDRESSTEXT'
 ).distinct()

Address_DF.count()

93401

In [51]:
#sometimes you will see error if you use the complete com.microsoft.sqlserver.jdbc.spark in the format deck.
#So just JDBC works in the higher versions of spark in the format space. 

Address_DF.write \
    .format("jdbc") \
    .mode("overwrite") \
    .option("url", url) \
    .option("dbtable", "AddressMaster_STG") \
    .option("user", username) \
    .option("password", password) \
    .save()

In [5]:
Sales_fact_DF= Sales_DF.select(
    'ID',
 'ORDERID',
 'ORDERDETAILID',
 'DATE_',
 'USERID',
 'STATUS_',
 'ITEMID',
 'AMOUNT',
 'TOTALPRICE',
 'ADDRESSID'
)


In [6]:
Sales_fact_DF = Sales_fact_DF.withColumn("Month",month("DATE_"))
Sales_fact_DF = Sales_fact_DF.withColumn("Year",year("DATE_"))

NameError: name 'month' is not defined

In [7]:
#sometimes you will see error if you use the complete com.microsoft.sqlserver.jdbc.spark in the format deck.
#So just JDBC works in the higher versions of spark in the format space. 

Sales_fact_DF.write \
    .format("jdbc") \
    .mode("overwrite") \
    .option("url", url) \
    .option("dbtable", "Sales_Fact_STG") \
    .option("user", username) \
    .option("password", password) \
    .save()

#<B>LETS WORK on some new libraries. </B>#

In [10]:
#DATAFRAMEWRITERV2
#lets divide the data by partition 
Sales_fact_DF2 = DataFrameWriterV2(Sales_fact_DF, table="Sales_Fact")

#lets write the data into parquet files in the folder.
Sales_fact_DF2=Sales_fact_DF2.write.format("parquet")

Sales_fact_DF2.mode("overwrite")\
.partitionedBy(months("DATE_"),years("DATE_"))\
.option("parquet.writer.compaction.code", "SNAPPY")\
.option("parquet.writer.compaction.threshold.bytes", 2 * 1024 * 1024)\
.save("D:/Data sets/Turkish_Market_Sales_Dataset/Spark_storage/")

AttributeError: 'DataFrameWriterV2' object has no attribute 'write'

In [15]:
#DATAFRAMEWRITERV2

Sales_fact_DF2 = DataFrameWriterV2(Sales_fact_DF, table="Sales_Fact")

#lets write the data into parquet files in the folder.
Sales_fact_DF2.option("Formtat","parquet")\
.option("writefile","D:/Data sets/Turkish_Market_Sales_Dataset/Spark_storage/V2/")\
.partitionedBy("DATE_")

In [27]:
###This is required when I have to stop the spark and restart the sessions. #####

sc = spark.sparkContext
sc.stop()
spark.stop()
print("server stopped")

server stopped
